In [ ]:
from os import listdir, path
import re

In [ ]:
BASE_DIR_PATH = path.dirname(path.abspath("__file__"))  # gdyby nie działało, usuń cudzysłów
INSTANCES_PATH = path.join(BASE_DIR_PATH, "instances")
NEGATIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "negative_random")
NEGATIVE_REPETITIONS_PATH = path.join(INSTANCES_PATH, "negative_repetitions")
POSITIVE_END_ERRORS_PATH = path.join(INSTANCES_PATH, "positive_end_errors")
POSITIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "positive_random")

In [ ]:
negative_random_filenames = [path.join(NEGATIVE_RANDOM_PATH, f) for f in listdir(NEGATIVE_RANDOM_PATH) if path.isfile(path.join(NEGATIVE_RANDOM_PATH, f))]
negative_repetitions_filenames = [path.join(NEGATIVE_REPETITIONS_PATH, f) for f in listdir(NEGATIVE_REPETITIONS_PATH) if path.isfile(path.join(NEGATIVE_REPETITIONS_PATH, f))]
positive_end_errors_filenames = [path.join(POSITIVE_END_ERRORS_PATH, f) for f in listdir(POSITIVE_END_ERRORS_PATH) if path.isfile(path.join(POSITIVE_END_ERRORS_PATH, f))]
positive_random_filenames = [path.join(POSITIVE_RANDOM_PATH, f) for f in listdir(POSITIVE_RANDOM_PATH) if path.isfile(path.join(POSITIVE_RANDOM_PATH, f))]

In [ ]:
selected_filenames = negative_random_filenames + negative_repetitions_filenames + positive_end_errors_filenames + positive_random_filenames

for filename in selected_filenames:
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]
        
        org_seq_len = re.search('(?<=\.)[0-9]+', inst_name).group(0) 
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = neg_errors_number.group(0) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = pos_errors_number.group(0) if pos_errors_number else 0
        
        print(inst_name)
        print(org_seq_len)
        print(neg_errors_number)
        print(pos_errors_number)
        print(file.read(10))